In [ ]:
import pandas as pd
import os, re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from hanspell import spell_checker
from eunjeon import Mecab
from tqdm import tqdm
import gensim
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
from soynlp.normalizer import *

## soynlp 학습

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

In [ ]:
corpus = DoublespaceLineCorpus("2016-10-20.txt")

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

# 유사도 사용해서 추천시스템 만들기

In [ ]:
# data = pd.read_csv(r'강의평.csv', encoding='cp949', index_col=0, low_memory=False).drop_duplicates(["강의평"])
# data.head()

In [ ]:
# 맞춤법 교정 (오래걸려서 따로 저장했으니 그거 열어서 사용하시면 됩니다)

# checked_sent = [spell_checker.check(sent).checked for sent in data['강의평'].tolist()]

# checked_sent = []
# for sent in data['강의평'].tolist():
#     spelled = spell_checker.check(sent)
#     res = spelled.checked
#     checked_sent.append(res)

# print(checked_sent[:3])

In [ ]:
# data['교정된 강의평'] = checked_sent

In [ ]:
data1 = pd.read_csv(r'강의평 수정.csv', encoding='cp949', index_col=0, low_memory=False).drop_duplicates(["교정된 강의평"])

In [ ]:
data1.head()

In [ ]:
data1["교정된 강의평"].isnull().sum()
data1 = data1.dropna(how='any')
print(data1.isnull().values.any())

In [ ]:
print(data1.shape)

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data1["교정된 강의평"].drop_duplicates())

print(tfidf_matrix.shape)

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(data.index, index=data["교정된 강의평"]).drop_duplicates()

print(indices.head())

In [ ]:
def get_recommendations(ques, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[ques]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['강의평'].iloc[movie_indices]

In [ ]:
get_recommendations("좋긴 하지만 과제가 은근 귀찮 교수님은 좋아요 근데 교양 외국어 학점 잘 받을 생각은 하지 마세요...")

# Doc2Vec을 활용한 강의 추천!

In [ ]:
mecab = Mecab()

In [ ]:
lec_eval = data1["교정된 강의평"].tolist()
lec_nme = data1["강의 이름"].tolist()
lec_pf = data1["교수"].tolist()

print(len(lec_eval))
print(lec_eval[:3])
print(len(lec_nme))
print(len(lec_pf))

In [ ]:
## 전처리
def normalization(lec_ev):
    pre_eval = []
    for text in lec_ev:
        temp = re.sub(r"[^가-힣ㅏ-ㅣㄱ-ㅎ0-9A-Za-z\~\+\-\♥\^\!\? ]", "", text)
        temp = re.sub(r"\<[a-z]+\/\>", "", temp)
        temp = emoticon_normalize(temp, num_repeats=2)
        pre_eval.append(temp)

    # 형태소 추출
    morphs_eval = [mecab.morphs(x) for x in pre_eval]

    return morphs_eval

morphs_eval = normalization(lec_eval)
print(morphs_eval[:5])

In [ ]:
# 불용어 제거

stop_words = ["은", "는", '이', '가', '아요', '입니다', '고', '으로', '다']

def stop_(morphs_eval, ques=False):
    merge_content = []
    for morph in morphs_eval:
        txt = ' '.join(morph)
        merge_content.append(txt)

    ### 불용어 리스트에 없는 단어들만 추가, 명사들로 이뤄진 리스트들
    texts = [[word for word in document.split() if word not in stop_words]
            for document in merge_content]
    
    if ques:
        return texts[0]

    return texts

texts = stop_(morphs_eval)

print(texts[:3])

In [ ]:
# 훈련하는데 오래걸리면 이 셀은 넘어가고

#corpus 읽어서 태그달기(태그는 임의의 태그, 입력된 순서대로 달림)
def read_corpus(texts):
    for i, content in tqdm(enumerate(texts)):
        yield gensim.models.doc2vec.TaggedDocument(content, [i])

corpus = list(read_corpus(texts))
print(texts[:3])
print("-"*100)
print(corpus[:3])
#모델 학습, doc2vec
model = gensim.models.doc2vec.Doc2Vec(vector_size=500, min_count=0, epochs=50, workers=4, dm=1, window=10) #모델 생성
model.build_vocab(corpus) #모델 훈련 위한 vocab 생성
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs) #모델 훈련

In [ ]:
model.save("eta_model.doc2vec")

# 위 셀 넘어갔으면 아래 코드 주석 풀고 실행
# model = Doc2Vec.load("eta_model.doc2vec")

In [ ]:
#main
query = input('궁금하신 사항을 입력해 주세요: ')
query = [query] #전처리에 인수 값으로 리스트 형식으로 넣어줘야 하기 때문에 문자열을 포함한 리스트 형식으로 변경

##전처리 진행, 전처리한 문장을 학습한 doc2vec모델에 적용해 vectorize
norm_content = normalization(query)
preprocessed = stop_(norm_content, ques=True)
print(preprocessed)
vector = model.infer_vector(preprocessed) #학습된 모델로 벡터값 유추

##가장 유사한 vector를 갖는 질문들 모아모아
sims = model.docvecs.most_similar([vector], topn=len(model.docvecs))

##유사도 상위 10개 tag들
sims_tags = list(map(lambda x: x[0], sims[:10])) 


print('\n\n')
for i, tag in enumerate(sims_tags):
    print(i+1, ')', '유사도: ', sims[i][1])
    print("유사한 강의평: %s"%(lec_eval[tag]))
    print('------질문------')
    print(query)
    print('\n')
    
    print('------강의명------')
    print(i+1, ")", lec_nme[tag])
    print('\n')
    
    print('------교수 이름------')
    print(i+1,')', lec_pf[tag])
    print('\n\n\n\n')